In [17]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import skimage, os
from skimage.morphology import ball, disk, dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import binary_dilation, binary_opening
from skimage.filters import roberts, sobel
from skimage import measure, feature
from skimage.segmentation import clear_border
from skimage import data
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import pydicom
import scipy.misc
import numpy as np
from skimage.measure import compare_ssim as ssim
from skimage.measure import compare_mse as mse

%matplotlib inline

def compare_images(original, processed):
    mse_noise = mse(original, processed)
    ssim_noise = ssim(original, processed, data_range=processed.max() - processed.min())
    print("SSIM = ", ssim_noise, "       MSE = ", mse_noise)

# Any results you write to the current directory are saved as output.
lung = pydicom.read_file('c:/temp/000074.dcm')

# convert to HounsField Scale
image = np.stack(lung.pixel_array)

# Convert to int16 (from sometimes int16), 
# should be possible as values should always be low enough (<32k)
image = image.astype(np.int16)

# Set outside-of-scan pixels to 1
# The intercept is usually -1024, so air is approximately 0
image[image == -2000] = 0
#image[(image<-500)|(image>500)] = 0

intercept = lung.RescaleIntercept
slope = lung.RescaleSlope
    
if slope != 1:
	image = slope * image.astype(np.float64)
	image = image.astype(np.int16)
        
image += np.int16(intercept)
    
imgs = np.array(image, dtype=np.float64)

#plt.imshow(imgs, cmap=plt.cm.bone)
#plt.figure()
#plt.hist(imgs, bins=20)
#plt.show()

##############################################################################################################
from skimage import exposure

p2, p98 = np.percentile(image, (2, 98))
img_rescale = exposure.rescale_intensity(image, in_range=(p2, p98))

compare_images(imgs, img_rescale)

##############################################################################################################

# Histogram Equalization
img_eq = exposure.equalize_hist(image)
compare_images(imgs, img_eq)

##############################################################################################################

# Adaptive Equalization
img_adapteq = exposure.equalize_adapthist(image, clip_limit=0.15)
compare_images(imgs, img_adapteq)

###############################################################################################################

#Denoise Chambolle
from skimage.restoration import (denoise_tv_chambolle, denoise_bilateral,
                                 denoise_wavelet, estimate_sigma)

img_chamboelle = denoise_tv_chambolle(image, weight=0.1, multichannel=False)
compare_images(imgs, img_chamboelle)

################################################################################################################

#Denoise Wavelet
img_wavelet = denoise_wavelet(image, multichannel=True)
compare_images(imgs, img_wavelet)

# Bilateral Denoise
img_bilateral = denoise_bilateral(image, sigma_color=0.1, sigma_spatial=15, multichannel=False)
compare_images(imgs, img_bilateral)

c:\users\hp\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: overflow encountered in short_scalars


SSIM =  0.011597795698408055        MSE =  697385995.418808
SSIM =  -6.850992479212441e-05        MSE =  394345.50598552055


c:\users\hp\appdata\local\programs\python\python35\lib\site-packages\skimage\util\dtype.py:126: UserWarning: Possible sign loss when converting negative image of type int16 to positive image of type uint16.
  .format(dtypeobj_in, dtypeobj_out))
c:\users\hp\appdata\local\programs\python\python35\lib\site-packages\skimage\util\dtype.py:130: UserWarning: Possible precision loss when converting from int16 to uint16
  .format(dtypeobj_in, dtypeobj_out))


SSIM =  -1.3127042723447138e-05        MSE =  394259.9781230515
SSIM =  1.074388630289893e-05        MSE =  394152.6464329034
SSIM =  1.0875560512068707e-05        MSE =  394152.02762699674


ValueError: Image must contain only positive values